In [ ]:
import datetime
import time

# 一、数据库


In [56]:
import pymysql

DATABASE_CONFIG = {
    'MYSQL_HOST' : 'localhost',
    'MYSQL_DBNAME' :  'weibo',
    'MYSQL_USER' : 'weibo',
    'MYSQL_PASSWD' : 'weibo',
    'MYSQL_PORT' : 3306
}

class DB:
    def __init__(self):
        self.connect = pymysql.connect(
            host = DATABASE_CONFIG['MYSQL_HOST'],
            db = DATABASE_CONFIG['MYSQL_DBNAME'],
            user = DATABASE_CONFIG['MYSQL_USER'],
            passwd = DATABASE_CONFIG['MYSQL_PASSWD'],
            charset = 'utf8mb4',
            use_unicode = True
        )
        self.cursor = self.connect.cursor()

    def getOneData(self):
        sql = "select * from weibo_filtered where id = 157485"
        self.cursor.execute(sql)
        return self.cursor.fetchone()
db = DB()
# oneItem = db.getOneData()

# 二、数据清理


## 1. 删除无效数据

In [ ]:
# 删除内容为空，或写数据时就存在的异常数据
delete_sql = "DELETE FROM weibo_origin WHERE created_at = 1 or content is null"
db.cursor.execute(delete_sql)
db.connect.commit()

## 2. 格式化时间字段和内容字段

- 时间
- 内容清理
    去html tag
    去emoji
    去@符等

In [ ]:
import emoji
import re

In [ ]:
UPDATE_NUM = 500
while True:
    sqlOne = "select id,created_at,content,source from weibo_origin where source_formated is null limit %s"%(UPDATE_NUM)
    db.cursor.execute(sqlOne)
    Items = db.cursor.fetchall()
    if Items:
        reg = re.compile('<[^>]*>') # 正则 去掉html tag
        df = []
        text_format = []
        source_format = []
        for item in Items:
            # print(item)
            result = reg.sub('',item[2]).replace('\n','').replace(' ','').replace("\'",'').replace("\\",'') #去tag
            result = emoji.demojize(result)  # 去emoji
            result = re.sub(':\S+?:', '', result)
            text_format.append(result)
            df.append(time.strftime("%Y-%m-%d %H:%M:%S", time.strptime(item[1], '%a %b %d %H:%M:%S +0800 %Y')))
            source = emoji.demojize(item[3])
            source = re.sub(':\S+?:', '', source).replace("\'",'')
            source = re.sub('\S+的','',source)
            source_format.append(source)
        sqlPartA = ''
        sqlPartB = ''
        sqlPartC = ''
        idList = []
        # 构建sql语句 批量更新 10条
        for i in range(len(Items)):
            tmpA = "WHEN %s THEN '%s' "%(Items[i][0],df[i])
            tmpB = "WHEN %s THEN '%s' "%(Items[i][0],text_format[i])
            tmpC = "WHEN %s THEN '%s' "%(Items[i][0],source_format[i])
            sqlPartA = sqlPartA + tmpA
            sqlPartB = sqlPartB + tmpB
            sqlPartC = sqlPartC + tmpC
            idList.append(str(Items[i][0]))
            
        update_sql = "update weibo_origin set created_timestamp = CASE id %s END,content_formated = CASE id %s END,source_formated = CASE id %s END where id IN (%s)"%(sqlPartA,sqlPartB,sqlPartC,','.join(idList))
        # print(update_sql)      
        db.cursor.execute(update_sql)
        db.connect.commit()
        if Items[0][0] % 1000 ==0:
            print(Items[0][0])
    else:
        break

# 三、数据挖掘

## 0. EDA

参见阿里Quick BI系统  
[公开地址](https://bi.aliyuncs.com/token3rd/dashboard/view/pc.htm?spm=a2c10.workspacedashboard.0.0.585e63adMR3i1C&pageId=1260e1a4-3045-475d-afc7-7b45e5a3f367&accessToken=6c38f132aa1927efc5396a22b0e2e205&qbi_version_param=1)

## 1. 分词

In [ ]:
# 使用jieba分词工具

In [ ]:
import jieba

sql = "select content_formated from weibo_filtered where id = 1707"
db.cursor.execute(sql)
res = db.cursor.fetchone()
seg_list = jieba.cut(res[0], cut_all=False)
print("/ ".join(seg_list))


## 2. TF-IDF

In [ ]:
import jieba.analyse
id = 0
while True:
    try:
        sql = "select content_formated from weibo_filtered where id > %s limit 50"%(id)
        db.cursor.execute(sql)
        res = db.cursor.fetchall()
        if res:
            jieba.suggest_freq('双十一', True)
            batch_insert = ""
            for (i,oneItem) in enumerate(res):
                tf_idf = jieba.analyse.extract_tags(oneItem[0], topK=20, withWeight=False, allowPOS=())
                for (index,word) in enumerate(tf_idf):
                    batch_insert = batch_insert + "('%s')"%word
                    if index != len(tf_idf) -1:
                        batch_insert = batch_insert +','
                if i != len(res) -1:
                    batch_insert = batch_insert +','
            insert_sql = "insert into key_word (word) values %s"%(batch_insert)
            db.cursor.execute(insert_sql)
            db.connect.commit()
            if id % 10000 ==0:
                print(id)
            id = id + 50

        else:
            break
    except Exception as error:
        print(error)
        print('--------'+str(id))



## 3. LDA 主题模型

## 4. 情感极性分析

使用HarvestText

In [1]:
from harvesttext import HarvestText
ht = HarvestText()

In [33]:
# result = db.getOneData()[4]
# print(result)
getData = "select content_formated from weibo_filtered limit 10000"
db.cursor.execute(getData)
res = db.cursor.fetchall()
docs = []
for i in res:
    docs.append(i[0])

https://www.zhihu.com/question/333074061/answer/736727068
移除表情符号后模型会更加从文本方面去理解和分类，避免过拟合，利于提高泛化性

In [34]:
sent_dict = ht.build_sent_dict(docs,min_times=1,scale="+-1")

In [59]:
print("%f:%s" % (ht.analyse_sent(result), result))
db = DB()

0.027812:双十一雨我无瓜


In [61]:

s_id = 0
while True:
    try:
        getData = "select id,content_formated from weibo_filtered where id > %s order by id asc limit 1000"%(s_id)
        db.cursor.execute(getData)
        res = db.cursor.fetchall()
        if res:
            sents = []
            ids = []
            sqlPart = ''
            for (index,oneItem) in enumerate(res):
                sent = ht.analyse_sent(oneItem[1])
                tmp = "WHEN %s THEN %s "%(oneItem[0],sent)
                sqlPart = sqlPart + tmp
                ids.append(str(oneItem[0]))
            
            update_sql = "update weibo_filtered set sentiment = CASE id %s END where id IN (%s)"%(sqlPart,','.join(ids))
            db.cursor.execute(update_sql)
            db.connect.commit()
            if s_id % 10000 ==0:
                print(s_id)
            s_id = s_id + 1000
        else:
            break
    except Exception as error:
        print(error)
        print(s_id)

0


KeyboardInterrupt: 

In [44]:
a = ht.analyse_sent('我每年都不关心双十一什么的购物节，感觉就省了不少心，当然我也不怎么太大批买东西……')
print(a)

0.23486331323938625
